# 🤖 LangChain Agents & Evaluation Demo

This notebook demonstrates **modern AI agent workflows** and **text evaluation metrics** using the latest LangChain 0.3.x APIs.

## 📋 What You'll Learn:

### 1. 📊 **Rouge Evaluation**
- Measure text similarity between reference and generated responses
- Understand precision, recall, and F1 scores for text generation quality
- Essential for evaluating RAG and chatbot responses

### 2. 🤖 **LangChain Agents** 
- **Legacy Approach**: Traditional `initialize_agent` (deprecated but educational)
- **Modern Approach**: Tool binding with `ChatOpenAI` and `@tool` decorators
- Robust error handling for API limitations and billing issues

### 3. 🛠️ **Tool Integration**
- Create custom tools for search, summarization, and other tasks
- Learn both synchronous and asynchronous tool execution patterns
- Understand tool descriptions and parameter validation

**💡 Note**: This notebook works in demonstration mode even without OpenAI API keys, making it perfect for learning and experimentation!

In [19]:
%pip install -q --upgrade rouge langchain langchain-openai \
        langchain-core langchain-community pydantic

Note: you may need to restart the kernel to use updated packages.


## 🔧 Environment Setup

**Important**: To run the agent examples with OpenAI, you need to set your API key:

```bash
export OPENAI_API_KEY="your-api-key-here"
```

Or create a `.env` file in your project root:
```
OPENAI_API_KEY=your-api-key-here
```

The examples below will work in demonstration mode even without an API key.

In [20]:
from rouge import Rouge

# Reference and generated responses
reference = ["Apache Airflow is a workflow orchestration tool."]
generated = ["Airflow is a tool for workflow orchestration."]

rouge = Rouge()
scores = rouge.get_scores(generated, reference)

print(scores)

[{'rouge-1': {'r': 0.8571428571428571, 'p': 0.8571428571428571, 'f': 0.8571428521428571}, 'rouge-2': {'r': 0.5, 'p': 0.5, 'f': 0.4999999950000001}, 'rouge-l': {'r': 0.7142857142857143, 'p': 0.7142857142857143, 'f': 0.7142857092857143}}]


In [22]:
import os
from langchain.agents import initialize_agent, Tool
from langchain_openai import OpenAI

# Check if API key is available
if not os.getenv("OPENAI_API_KEY"):
    print("⚠️  OpenAI API key not found. Setting a placeholder for demonstration.")
    os.environ["OPENAI_API_KEY"] = "sk-placeholder-key"

# Define tools
def search_tool(query):
    # Simulate a search tool
    return f"Search results for: {query}"

tools = [
    Tool(name="search", func=search_tool, description="Performs a web search.")
]

try:
    # Initialize the LLM (using a mock approach if no valid API key)
    llm = OpenAI(model="gpt-3.5-turbo-instruct", openai_api_key="sk-placeholder")
    
    print("🤖 Creating agent with legacy approach (deprecated but functional)...")
    # Create an agent
    agent = initialize_agent(
        tools, llm, agent="zero-shot-react-description", verbose=True)
    
    # Instead of calling OpenAI API, let's demonstrate the structure
    print("📋 Agent created successfully!")
    print("🔧 Tools available:", [tool.name for tool in tools])
    print("⚡ Agent type: zero-shot-react-description")
    print("\n💡 Note: This would normally call OpenAI API, but we're showing structure instead.")
    
except Exception as e:
    print(f"❌ Error creating agent: {e}")
    print("💡 This is likely due to API key issues. See the modern approach below for better error handling.")

🤖 Creating agent with legacy approach (deprecated but functional)...
📋 Agent created successfully!
🔧 Tools available: ['search']
⚡ Agent type: zero-shot-react-description

💡 Note: This would normally call OpenAI API, but we're showing structure instead.


#### Modern LangGraph Agent Approach (Recommended)
Using the new LangGraph for more robust agent workflows.

In [23]:
import os
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

@tool
def search_tool(query: str) -> str:
    """Performs a web search and returns results."""
    # Simulate a search tool with more realistic response
    return f"🔍 Search results for '{query}':\n1. AI advances in 2024 show remarkable progress\n2. New machine learning breakthroughs announced\n3. Industry reports on AI adoption trends"

@tool  
def summarize_tool(text: str) -> str:
    """Summarizes the given text."""
    return f"📄 Summary: {text[:100]}... (Key insights: Latest AI developments show continued innovation)"

# Check for API key
api_key = os.getenv("OPENAI_API_KEY")
if not api_key or api_key == "sk-placeholder-key":
    print("⚠️  No valid OpenAI API key found.")
    print("🔧 Demonstrating tool structure without API calls...")
    
    # Show how tools work without API
    tools = [search_tool, summarize_tool]
    
    print("\n🛠️  Available Tools:")
    for tool in tools:
        print(f"  • {tool.name}: {tool.description}")
    
    # Demonstrate tool usage
    query = "Find the latest news about AI and summarize it"
    print(f"\n📝 Query: {query}")
    
    # Simulate tool calls
    search_result = search_tool.invoke({"query": "latest AI news"})
    print(f"\n🔍 Search Tool Result:\n{search_result}")
    
    summary_result = summarize_tool.invoke({"text": search_result})
    print(f"\n📄 Summary Tool Result:\n{summary_result}")
    
else:
    try:
        # Initialize the LLM with modern approach
        llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)
        
        # Tools list
        tools = [search_tool, summarize_tool]
        
        # Bind tools to the model (modern approach)
        llm_with_tools = llm.bind_tools(tools)
        
        # Make a query
        query = "Find the latest news about AI and summarize it."
        messages = [HumanMessage(content=query)]
        
        print(f"🤖 Processing query: {query}")
        print("⚡ Using modern LangChain tool binding...")
        
        # Get response with tool calls
        response = llm_with_tools.invoke(messages)
        print(f"\n🧠 LLM Response: {response.content}")
        
        # Check if there are tool calls
        if hasattr(response, 'tool_calls') and response.tool_calls:
            print(f"\n🔧 Tool calls detected: {len(response.tool_calls)}")
            for i, tool_call in enumerate(response.tool_calls, 1):
                print(f"\n  {i}. Tool: {tool_call['name']}")
                print(f"     Args: {tool_call['args']}")
                
                # Execute the appropriate tool
                if tool_call['name'] == 'search_tool':
                    result = search_tool.invoke(tool_call['args'])
                    print(f"     Result: {result}")
                elif tool_call['name'] == 'summarize_tool':
                    result = summarize_tool.invoke(tool_call['args'])
                    print(f"     Result: {result}")
        else:
            print("\n💭 No tools were called - LLM provided direct response")
            
    except Exception as e:
        print(f"❌ Error with OpenAI API: {e}")
        print("💡 This might be due to API limits, billing, or network issues.")
        print("🔄 Falling back to tool demonstration mode...")
        
        # Fallback to demonstration mode
        search_result = search_tool.invoke({"query": "latest AI news"})
        summary_result = summarize_tool.invoke({"text": search_result})
        print(f"\n🔍 Demo Search: {search_result}")
        print(f"📄 Demo Summary: {summary_result}")

print("\n✅ Modern agent demonstration complete!")
print("💡 In production, consider using LangGraph for more complex agent workflows.")

🤖 Processing query: Find the latest news about AI and summarize it.
⚡ Using modern LangChain tool binding...
❌ Error with OpenAI API: Error code: 429 - {'error': {'message': 'Your account is not active, please check your billing details on our website.', 'type': 'billing_not_active', 'param': None, 'code': 'billing_not_active'}}
💡 This might be due to API limits, billing, or network issues.
🔄 Falling back to tool demonstration mode...

🔍 Demo Search: 🔍 Search results for 'latest AI news':
1. AI advances in 2024 show remarkable progress
2. New machine learning breakthroughs announced
3. Industry reports on AI adoption trends
📄 Demo Summary: 📄 Summary: 🔍 Search results for 'latest AI news':
1. AI advances in 2024 show remarkable progress
2. New machin... (Key insights: Latest AI developments show continued innovation)

✅ Modern agent demonstration complete!
💡 In production, consider using LangGraph for more complex agent workflows.
❌ Error with OpenAI API: Error code: 429 - {'error': {'m

## 🎯 Summary

This notebook demonstrates:

1. **📊 Rouge Evaluation**: Measuring text similarity between reference and generated responses
2. **🤖 Legacy Agents**: Traditional LangChain agent approach (deprecated but educational)  
3. **⚡ Modern Tool Binding**: Current best practices with error handling and fallbacks
4. **🛠️ Robust Implementation**: Works even without valid API keys for learning purposes

### 🚀 Next Steps for Production:

- Set up proper OpenAI API keys and billing
- Consider **LangGraph** for complex multi-agent workflows  
- Implement **persistent memory** for longer conversations
- Add **custom tools** for your specific use case
- Use **streaming** for real-time responses

### 📚 Learn More:

- [LangChain Agent Documentation](https://python.langchain.com/docs/modules/agents/)
- [LangGraph for Advanced Workflows](https://langchain-ai.github.io/langgraph/)
- [Tool Creation Guide](https://python.langchain.com/docs/modules/tools/)